In [2]:
import numpy as np
import pandas as pd
import re
import pickle
from pprint import pprint


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import json
data = json.loads('{"one" : "1", "two" : "2", "three" : "3"}')

from bson import json_util

from mongoengine import *

In [5]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

restautant_id = 'BNGHSR0001'

final_list_json=[]

In [6]:

for n,main_cat_name in enumerate(str_menu.loc[:,'main_category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=main_cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['sub_category'] = []
    for p,sub_cat_name in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category'].unique()):
        final_list_json[n]['sub_category'].append({})
        final_list_json[n]['sub_category'][p]['name']=sub_cat_name
        final_list_json[n]['sub_category'][p]['description']=''
        final_list_json[n]['sub_category'][p]['priority']=''
        final_list_json[n]['sub_category'][p]['foodlist']=[]
        for q,food_dict in enumerate(str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name][str_menu[str_menu['main_category'].loc[:'sub_category']==main_cat_name].loc[:,'sub_category']==sub_cat_name].iterrows()):
            final_list_json[n]['sub_category'][p]['foodlist'].append({})
            final_list_json[n]['sub_category'][p]['foodlist'][q]['name']= food_dict[1]['dish_name']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['description']=food_dict[1]['description']
            final_list_json[n]['sub_category'][p]['foodlist'][q]['price']=food_dict[1]['price']
        
        

KeyError: 'dish_name'

# Pages

In [3]:
pages={}

In [4]:
pages[0]="""$BAR BITES

NACHOS (Veg/Chicken)

Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream 

190/230

CHEESE MASALA PAPAD

Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese

160

H.O.C SIGNATURE MASALA PEANUTS

A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices 

160

SPICY TRICOLOUR CHAAT

Fresh Peas, Corn Kernels & Black Gram Tossed With Chillies & Sirka onions

180

CHATPATA CHANA CHAAT

A Delectable Blend Of Bengal Gram And A Tangy Mixture

160

TANDOORI CHICKEN CHAAT

Clay Oven Roasted Chicken Blended With Classic Spices

220

CHEESE DYNABITES

Classic Cheese Balls Served with New Orleans Dip

220

$SALADS

GREEN SALAD

Traditional Green Cuts With Mint Yoghurt Dip

170

HOC SALAD (Veg/Chicken)

Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing

180/210

WATERMELON & FETA

An All Time Classic - With Balsamic Glaze

210

CAESAR SALAD (Veg/Chicken)

Crunchy Ice Berg Lettuce, Parmesan With the Classic Caesar Dressing

190/230"""

In [5]:
pages[1]="""$VEG STARTERS

CHEESE & CORN TRIANGLE (new)

Shredded Cheese, Corn & Seasonings Wrapped In Wonton Sheet & Deep Fried

220

CHAMPIGNONS FARCIS (new)

Sauteed Button Mushroom Stuffed With Cheese & Herbs

230

CHILLI CHEESE CIGARA (new)

Deep Fried Rolls With Cheese, Chill & Herbs

220

HOC CHIP & DIP BASKET

Combination Of Onion Rings, Jalapeno Cheese Balls, Greek Potato Wedges, Habanero Green Chili Fries

390

FRENCH FRIES

French Fries Tossed With Hoc Signature Cajun Spice Seasoning

190

ONION RINGS

Extra Crispy Onion Rings With A Spicy Salsa

190

MUSHROOM LOADED FRIES

Cheesy French Fries Topped With Mushrooms And Herbs

230

GARLIC BREAD

Garlic Butter Flavoured Bread Topped With Cheese

190

CHEESE CORN TIKKI

American Com & Cottage Cheese Patty Served with Mint Chutney

220

CRISPY CHILLY MUSHROOM

Crispy Fried Mushrooms Tossed With Oriental Spices

220

CRISPY CHILLY BABY CORN

Crispy Fried Babycom Tossed With Oriental Spices

190

CRISPY CORN CHILLY PEPPER

Fred Undi Lightly Crispy & Seasoned With Lime Juice, Salt, Chilli & Pepper

240

HONEY CHILLY POTATO

Crispy Fried Potatoes Tossed With Chilli And Honey

190"""

In [6]:
pages[2]="""$VEG STARTERS

SESAME CHILLI PANEER

Cottage Cheese Cubes Tossed In A Sweet & Spicy Sauce, Finished With Sesame Seeds

270

PANDAN LEAF COTTAGE CHEESE

Tender Cottage Cheese Wrapped In Fragrant Pandan Leaves, With Chilli Coriander Sauce

280

MEZZE BASKET

A Combination Of Hummus, Mutabal & Labneh, Served With Pita Bread

340

$FROM THE CLAY OVEN

MAKHMALI PANEER TIKKA

Mouth Melting Paneer Cubes Marinated With A Cheese Curd Mixture, Cream And Mild Spices

270

ANGARI PANEER TIKKA

Clay Oven Roasted Spicy Morsels Of Cottage Cheese

270

TANDOORI STUFFED MUSHROOMS

Mushrooms Filled With A Blend Of Indian Spices And Tangy Stuffing Mix, Adding A Unique Taste On The Bite

250

TANDOORI VEG MOMOS

Soft And Juicy Vegetable Momos Marinated With Tandoori Masala And Char Grilled

220

$CHICKEN STARTERS

CHICKEN LOADED FRIES

Cheesy French Fries Topped With Chicken & Herbs

260

SHANG DONG CHICKEN LOLLY POP

Crispy Lolly pops Crunched To Perfection & Tossed in our Signature Shang Dong Drip

260"""

In [7]:
pages[3]="""$CHICKEN STARTERS

HOC SIGNATURE CHICKEN WINGS

Sweet Chilly / Spiced Szechuan

260

PEPPER GARLIC CHICKEN

Cubes Of Chicken Tossed With Garlic And Pepper In Oriental Style

260

GUNTUR JUNCTION CHICKEN

Chicken Cubes Tossed With Freshly Ground Red Spices And Finished In South Indian Style

270

ANDHRA CHILLI CHICKEN *VERY HOT*

All Time Favourite Andhra Style Chicken Cooked With Coriander And Green Chilli Paste And Fresh Green Chillies

270

LEMON PEPPER CHICKEN

Slice Chicken Wok Tossed, Finished With Pepper Garlic & Lime

260

JIANG CHILLI CHICKEN

Chicken Tossed With Shredded Ginger And Chilli In Jiang Style

260

CHICKEN GHEE ROAST

Chicken Roasted With A Fiery Red, Tangy And Spicy With The Flavour Of Ghee

280

MASALA FRY CHICKEN (new)

Classic Indian Style With Bone Deep fried Chicken Served With Mint Chutney

250

$FROM THE CLAY OVEN

JAIPURI MURGH KA SOOLA

Succulent Pieces Of Chicken Zapped With Red Spices, Cinnamon Flavour & Finished In Tandoor

270

TANDOORI MURGH (HALF/FULL)

Chicken Marinated In Aromatic Indian Spices & Then Cooked In Tandoor

250/480

TANDOORI POMFRET

Indian Spices Marinated Whole Pomfret Skewered And Finished In Clay Oven

590"""

In [8]:
pages[4]="""$FROM THE CLAY OVEN

FISH TIKKA KALIMIRCH

Fresh Cubes Of Basa Fish Marinated With Crushed Black Pepper & Indian Spices, Finished In Clay Oven

320

AMRITSAR FISH TIKKA

Spicy Fish Tikka Marinated In Traditional Spices And Char Grilled

320

KALMI KEBAB (3pcs)

Clay Oven Roasted Chicken Drumsticks

260

ANGARI MURGH TIKKA

Clay Oven Roasted Spicy Morsels Of Chicken

260

TANDOORI LOLLIPOPS

Chicken Lollipops Marinated With Aromatic Indian Spices And Cooked In The Tandoor

270

MURGH MALAI TIKKA

Clay Oven Roasted Creamy Chunks Of Chicken

260

MURGH KASTURI KEBAB

Boneless Charcoal Smoked Chicken Tikka With Kasturi Methi Flavoured

260

TANDOORI CHICKEN MOMOS

Soft And Juicy Chicken Momos Marinated With Tandoori Masala And Char Grilled

260

TANDOORI KING PRAWNS

King Prawns Marinated With Yogurt, Lime Juice, Bhatti Spices And Slowly Cooked In Tandoor

499

$LAMB STARTERS

ANDHRA LAMB PEPPER FRY

Tender Mutton Pieces Tossed In South Indian Spices

350

MUTTON GHEE ROAST

Mutton Roasted With A Fiery Red And Spicy With The Flavour Of Ghee

370"""

In [9]:
pages[5]="""$LAMB STARTERS

LAHORI SEEKH KEBAB

Succulent Minced Lamb Kebabs In The True Lahori Street Food Style

370

RAILWAY MUTTON DRY

With Bone Mutton Cooked In Rich Indian Spices, As Served In The Railway Canteen Of The British Raj

350

$EGG STARTERS

EGG BHURJI

Desi Style Scrambled Eggs

170

SPICY MASALA OMELETTE

Slow Cooked Egg Omlet With Masala

180

EGG CHILLI 

Spicy Boiled Egg With Chilli Fry

210

$FROM THE GRILL

PERI PERI PANEER/CHICKEN

Peri Peri Basted Paneer / Chicken Cubes Served With Tzatziki Dip

260/280

THAI GRILLED FISH IN BANANA LEAF

Delicately Grilled Fish With Basil Flavour And Served Hot

330

HARISSA GRILLED FISH

Fish Marinated In A Spicy Tunisian Harissa Sauce And Grilled To Perfection

330

THAI GRILLED CHICKEN

Boneless Chicken Marinated With Thai Spices And Grilled

270"""

In [10]:
pages[6]="""$SEAFOOD STARTERS

PAN FRIED CHILLY FISH

Pan Fried Fish Tossed With Shredded, Onion, Capsicum And Green Chilli

330

THAI PEPPER GARLIC PRAWNS

Prawns Prepared In Thai Style With A Flavour Of Lemon Grass

380

PRAWNS IN RCP SAUCE

Wok Tossed Prawns Cooked With Roasted Chilli Paste And Seasonings

380

H.O.C FISH FINGERS

Classic Crumb fried Served with Tartar Sauce

330

ASIAN STYLE JUMBO PRAWNS

Tsing Hol Style / Singapore Chilli Garlic Style

499

$SANDWICHES/BURGERS

TOASTED VEG CLUB SANDWICH

Double Decker Layered With Lettuce, Cucumber, Cheese, Tomato And Served With Fries

240

TOASTED CHICKEN CLUB SANDWICH

Double Decker Layered With Lettuce, Cheese, Chicken, Fried Egg And Served With Fries

270

CLASSIC BURGERS (VEG / CHICKEN / LAMB)

Tender And Juicy Veg, Chicken Or Lamb Patties, Grilled And Served Fries With

250/280/300

HOC BURGER- THE BIG ONE

Double Chicken, Double Cheese And Topped With A Sunny Side Up Fried Egg, Served With Fries

350

$THE GRILL SIZZLERS

MIXED VEGETABLE SIZZLER

Grilled Vegetable Patties, Cottage Cheese And English Vegetables, Served With French Fries

350"""

In [11]:
pages[7]="""$THE GRILL SIZZLERS

ROSEMARY SCENTED COTTAGE CHEESE STEAKS

Cottage Cheese Steaks Wth Bell Peppers, Sage Pilaf With Grilled Vegetables & Jalapeno Tomato Coulis

380

CHICKEN CAFREAL

Authentic Goan Herb Marinated Chicken, Pan Fried, Stewed And Served With Potato Wedges & Sauteed Vegetables

390

CHICKEN SIZZLER

Chicken with red wine sauce, served with saute vegetable and french fries

380

TEQUILA LIME GRILLED CHICKEN

Flesh Flamed Tequila Marinated Lime & Chilli Chicken With Herbed Country Mash, Garden Grilled Vegetables Cumin Coffee Scented Jus

400

GRILLED FISH

Grilled Fish Fillet, With Lemon butter sauce Served With Herb Rice & saute vegetables

400

CAPTAIN MORGAN FLAMED LAMB

Rum Infused Caribbean Lamb & Pine Nut Patty With A Centred Egg, Charred Peppers, Jalapeno Aloll, Smoked Onion Marmalade, English Cheddar Melt, French Fries With A Sage & Onion

440

$PASTAS

BAKED CHEESE MACARONI

All Time Favourite In A Cheesy Cream Sauce And Baked in The Oven

300

SPAGHETTI AGLIO OLIO (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380

FIERY PENNE PASTA (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380

SPICY CAJUN LINGUINE ALFREDO (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380

PENNE WITH PESTO (VEG/ CHICKEN / SHRIMPS)

NA

300/340/380"""

In [12]:
pages[8]="""$PIZZAS

MARGHERITA

An All-Time Favourite Of Pomodoro, Water Buffalo Mozzarella And Basil Chiffonade

290

FUNGHI

Shitake, Oyster & Button Mushrooms With Shallots, Chives, Charmed Peppers And White Truffle Oil

300

PANEER TIKKA & CHARRED PEPPERS

Tandoori Paneer Mirch Tikka With Green Bell Peppers, Onion roundels, Spring Greens & Mint Coulis

350

MEDITERRANEAN GARDEN

Organic Greens With Imported FETA Cheese, Bell Peppers, Zucchini, Asparagus Spears, Artichokes & Sun Dried Tomatoes

360

TANDOORI MURGH TIKKA WITH KALAMATA OLIVES

Chicken Tikka With Shallots, Capers, Charred Peppers, Kalamata Olives & Sweet Chilli Treacle

380

BARBEQUED GERMAN CHICKEN SAUSAGES

BBQ Glazed Chicken Sausages With Caramelized Shallots, Bell Peppers, Jalapenos & Capers

390

MUTTON MASALA KEEMA PIZZA

Minced Mutton Cooked In Indian Spices & Herbs Topped With Chillies & Assorted Bell peppers

390

FRUITS OF THE SEA

Fruits Of The Sea - Calamari, Shrimps, Sea Bream Dices Served With Maple Butter & Fresh Red Chile

390

MIXED MEAT MAGIC

Age Old English Cheddar Topped With Roast Chicken Slvers, Apple Glazed Chicken Ham, Lamb Mince, Calamari & Shrimps

420"""

In [13]:
pages[9]="""$MAIN COURSE

FISH AND CHIPS

Crumb Fried Fish In Hoc Signature Marination, Served With French Fries

350

CHICKEN BASKET

Crumb Fried Chicken Supreme In Hoc Signature Marination, Served With French Fries

350

THAI CURRY [Red/Green] 

Basil Flavoured Thal Curry/bird-eye Chillies Flavoured Thai Curry Cooked With Garden Fresh Vegetables/chicken/prawns, Served With Steamed Rice

250/280/300

CHINESE CHOPSUEY

Vegetables/chicken Tossed In Celery Sauce, Served On A Bed Of Crispy Noodles

280/300

AMERICAN CHOPSUEY

Vegetables/chicken Tossed In A Sweet & Sour Sauce, Served On A Bed Of Crispy Noodles

280/300 

EXOTIC VEGETABLES WITH SAUCE OF YOUR CHOICE

Hot Garlic, Chili Basil, Sichuan

250

CHICKEN WITH SAUCE OF YOUR CHOICE

Hunan, Sichuan, Black Pepper

280

BASA FISH/PRAWNS WITH CHOICE OF YOUR CHOICE

Ginger Chilli, Sichuan, Pepper Garlic

270/300

TIKKA MASALA GRAVY

Paneer/chicken Cooked In A Spicy Onion Tomato Base Gravy

250/280

PALAK PANEER

Cottage Cheese Cooked In Spinach Puree, Tempered Cumin And Ginger Garlic Paste

260

DAL MAKHANI

Slow Cooked Lentils Finished With White Butter

240"""

In [14]:
pages[10]="""$MAIN COURSE

MAKHANI GRAVY

Paneer/Chicken Cooked In A Creamy Tomato Base Gravy

250/280

KHUMB HARA PYAZA

Vintage Mushrooms & Spring Onions Cooked To Perfection

270

DAL TADKA

Yellow Lentil Tempered With Garlic And Cumin Seeds, Punished With Chopped Onion And Tomato

190

BUTTER CHICKEN

All-Time Favorite

280

KADAI VEG/CHICKEN

Mix Veg/Chicken Pleces Tossed With Onion, Capsicum And Garnished With Onion Tomato Gravy

250/270 

MURGH METHI KHAS

Tender Chicken Cooked In Cream Based Onion And Tomato Gravy With Aromatic Spices And Flavor

280

DUM KA GHOST

Slow Cooked Mutton In A Spicy Onion & Tomato Gravy

330

$RICE & NOODLES

STIR FRIED RICE

(veg/egg/chicken/prawns)

200/220/250/280

STIR FRIED NOODLES

(veg/egg/chicken/prawns)

200/220/250/280

SICHUAN FRIED RICE

(veg/egg/chicken/prawns)

200/220/250/280

SICHUAN FRIED NOODLES

(veg/egg/chicken/prawns)

200/220/250/280

CHILLY GARLIC NOODLES

(veg/egg/chiden/prawns)

230/250/270/290"""

In [15]:
pages[11]="""$ACCOMPANIMENTS

JEERA RICE

NA

160

STEAMED RICE

NA

130

ROTI PLAIN/BUTTER

NA

35/40

KULCHA PLAIN/BUTTER

NA

35/40

NAAN PLAIN/BUTTER

NA

40/45

GARLIC NAAN

NA

50

LACCHA PARATHA

NA

45

$DESSERTS

GULAB JAMUN

Cardamom Flavoured Milk Dumplings Soaked In Sugar Syrup

90

WALNUT BROWNIE

Warm Chocolate Brownie Served With Vanilla ice cream

170

CARAMEL CUSTARD

Custard Dessert With A Layer Of Soft Caramel Glazed On Top

150

CHOICE OF ICE CREAMS-2 Scoops

Ask for the Flavours

90"""

# Consolidating food menu pages

In [17]:
full_menu_pages=[]

for i in range(12):
    full_menu_pages.append(pages[i])

full_menu_text="\n\n".join(full_menu_pages)

In [18]:
structured_menu = pd.DataFrame(columns=['category','name','description','price'])

description=dish_name=price=subcat=''
counter=0
for line in full_menu_text.split("\n\n"):
    if(re.search("^[\$]",line)):
        subcat = line[1:].strip()
    else:
        if(counter%3==0):
            dish_name = line.strip()
        elif(counter%3==1):
            description = line.strip()
        elif(counter%3==2):
            price = line.strip()
            structured_menu.loc[structured_menu.shape[0]]=[subcat,dish_name,description,price]
        counter+=1

In [19]:
str_menu=structured_menu.copy()

In [21]:
str_menu.loc[27,"category"]="VEG FROM THE CLAY OVEN"

str_menu.loc[28,"category"]="VEG FROM THE CLAY OVEN"

str_menu.loc[29,"category"]="VEG FROM THE CLAY OVEN"

str_menu.loc[30,"category"]="VEG FROM THE CLAY OVEN"

str_menu.loc[str_menu['category']=='FROM THE CLAY OVEN',"category"]="NON-VEG FROM THE CLAY OVEN"

In [22]:
str_menu.shape

(128, 4)

In [27]:
pickle.dump(str_menu,open('pkls/full_menu_main.pkl','wb'))

In [28]:
str_menu=pickle.load(open('pkls/full_menu_main.pkl','rb'))

In [25]:
final_list_json=[]

for n,cat_name in enumerate(str_menu.loc[:,'category'].unique()):
    final_list_json.append({})
    final_list_json[n]['name']=cat_name
    final_list_json[n]['description'] = ''
    final_list_json[n]['priority']=''
    final_list_json[n]['food_list'] = []
    for p,food_dict in enumerate(str_menu[str_menu['category']==cat_name].iterrows()):
        final_list_json[n]['food_list'].append({})
        final_list_json[n]['food_list'][p]['name']= food_dict[1]['name']
        final_list_json[n]['food_list'][p]['description']=food_dict[1]['description']
        final_list_json[n]['food_list'][p]['price']=food_dict[1]['price']
        

In [29]:
pickle.dump(final_list_json,open('pkls/final_usable_json.pkl','wb'))

final_list_json=pickle.load(open('pkls/final_usable_json.pkl','rb'))

# Bar Pages

In [31]:
bar_pages = {}

In [32]:
bar_pages[0] = """$ MARTINIS

CLASSIC MARTINI 
GIN & CHOICE OF VERMOUTH
290


APPLE & CUCUMBER MARTINI 
VODKA GREEN APPLE & CUCUMBER
290 


CHILLI BASIL MARTINI 
VODKA, BASIL, CHILLI & FRESH LIME
290 

WHITE LADY
GIN, TRIPLE SEC, SWEET & SOUR
290


ESPRESSO MARTINI 
SMIRNOFF ESPRESSO & KAHLUA
290 



WATERMELON BASIL MARTINI 
FRESH WATERMELON, BASIL & VOOKA 
290

TURKISH CHOCOLATINI
VODKA, CHOCOLATE MADE EXOTIC WITH A MINT OF ROSE
290


ORANGE MARMALADE MARTINI 
VODKA, ORANGE, MARMALADE
290 






$ CLASSICS

COSMOPOLITAN 
VODKA ,TRIPLE SEC & CRANBERRY JUICE
350



SEX ON THE BEACH 
VODKA, PEACH, ORANGE & CRANBERRY
420/620

TEQUILA SUNRISE 
TEQUILA, ORANGE JUICE & GRENADINE
390

PINA COLADA 
WHITE RUM, COCONUT & PINEAPPLE
350

SCREWDRIVER 
VODKA & ORANGE JUICE
370

MAI TAI
WHITE RUM, DARK RUM, TRIPLE SEC, ALMOND SYRUP, ORANGE & PINEAPPLE
350/520

WHISKY SOUR
WHISKY, FRESH LIME & SUGAR
390

LONG ISLAND ICED TEA 
VODKA, GIN, RUM, TEQUILA, TRIPLE SEC & COKE
420/620

MARGARITA 
REGULAR/FLAVOUR /FROZEN STRAWBERRY, LITCHI, GREEN APPLE, ORANGE, KIWI. ASK FOR YOUR CHOICE
390

MOJITO 
ASK FOR YOUR FLAVOUR. WHITE RUM, FRESH MINT, SUGAR FRESH LIME & SODA
370

CAIPIROSKA 
ASK FOR YOUR FLAVOUR. VODKA, FRESH LIME & SUGAR
350

SANGRIA RED/WHITE 
WHITE/RED WINE, TRIPLE SEC, BRANDY & DICED FRUITS
370/520

BLOODY MARY 
VODKA, TOMATO JUICE, WORCESTERSHIRE, TABASCO & FRESH LIME
370

"""

In [33]:
bar_pages[1]="""

$SIGNATURE

RED GIANT
VODKA, BACARDI LEMON, THYMES WATERMELON JUICE, PEACH & SPRITE 
350/590

BANGALORE SUNSET 
VODKA, RUM, GIN, TEQUILA, RED WINE, PEACH, LEMONADE
420/620 

ELECTRIC LEMONADE 
GIN, VODKA, RUM, BLUE CURACAO, LEMONADE
390/590 

AON KIWI 
WHITE RUM, FRESH KIWI, PINEAPPLE JUICE, LIME JUICE & EGG WHITE
370 


HOC SPECIAL MARGARITA 
TEQUILA, PASSION FRUIT, LIME, BEER
420


HELLO YELLOW
VODKA, ORANGE JUICE, PEACH & APRICOT, MINT SPRIGS & FRESH LIME 
350

LOVE BITES 
WHITE RUM, PINEAPPLE JUICE, LIME JUICE, BITTER GOURD, ORANGE OIL, CUCUMBER, LEMONGRASS SYRUP
390


$ BIG GUNS

LIIT PITCHER 
VODKA, GIN, RUM, TEQUILA, TRIPLE SEC, COKE
1250

BEER MOJITO PITCHER 
WHITE RUM, LEMON, MINT SPRIGS, BEER
1050 

ELECTRIC LEMONADE PITCHER 
VODKA, RUM, GIN, BLUE CURACAO, SPRITE
1150

BEACH ICE TEA PITCHER 
VODKA, RUM, GIN, TEQUILA, TRIPLE SEC, CRANBERRY
1190

SANGRIA PITCHER 
RED WINE, BRANDY, FRESH CHOPPED FRUITS
1090 

HOC SUNSET PITCHER
VODKA, RUM,GIN, TEQUILA, RED WINE, PEACH, LEMONADE
1190

$ SUN DOWNER’S

SPIN FLIP 
VODKA FRESH LEMON GRASS, GINGER, LEMON GRASS SYRUP,ORANGE OIL SWEET & SOUR
370

LOBLOLLY BAY 
WHITE RUM, PINEAPPLE JUICE, BASIL LEAVES,MAPLE SYRUP, VANILLA & LIME JUICE
370


SHAMROCK SHILLELAGH 
WHISKY, BAILEYS, KAHLUA, HAZELNUT SYRUP, ESPRESSO & FRESH CREAM 
390

BULL FROG 
VODKA, GIN, TEQUILA RUM, BLUE CURACAO, RED BULL
420/620

GREEN HORN 
GIN, AQUILA LEAVES, SPINACH, PINEAPPLE JUICE, HOMEMADE CUMIN BITTERS,SWEET & SOUR
350

RED VELVET 
GIN, FRESH POMEGRANATE, GRENADINE.FRESH BASIL, SWEET & SOUR
350
"""

# Consolidating bar menu pages

In [34]:
partial_bar_menu = []
for key in bar_pages.keys():
    partial_bar_menu.append(bar_pages[key])

In [35]:
bar_items=re.split("[\n]{2,7}","\n".join(partial_bar_menu))

In [36]:

structured_bar_menu = pd.DataFrame(columns=['category','name','description','price'])
sub_category = ''
description=bar_item_name=price=subcat=''
for item in bar_items:
    if(re.search("^\$",item)):
        sub_category=item[1:].strip()
    else:
        bar_item_name=item.split('\n')[0]
        description=item.split('\n')[1]
        price = item.split('\n')[2]
        structured_bar_menu.loc[structured_bar_menu.shape[0]]=[sub_category,bar_item_name,description,price]

In [45]:
pickle.dump(structured_bar_menu,open('pkls/partial_bar_menu.pkl','wb'))

In [46]:
bar_str_menu=pickle.load(open('pkls/partial_bar_menu.pkl','rb'))

In [48]:
bar_final_json=[]

bar_final_json=[]
for n,cat_name in enumerate(bar_str_menu.loc[:,'category'].unique()):
    bar_final_json.append({})
    bar_final_json[n]['name']=cat_name
    bar_final_json[n]['description'] = ''
    bar_final_json[n]['priority']=''
    bar_final_json[n]['food_list'] = []
    for p,food_dict in enumerate(bar_str_menu[bar_str_menu['category']==cat_name].iterrows()):
        bar_final_json[n]['food_list'].append({})
        bar_final_json[n]['food_list'][p]['name']= food_dict[1]['name']
        bar_final_json[n]['food_list'][p]['description']=food_dict[1]['description']
        bar_final_json[n]['food_list'][p]['price']=food_dict[1]['price']
        

In [49]:
pickle.dump(bar_final_json,open('pkls/bar_final_json.pkl','wb'))

bar_final_json=pickle.load(open('pkls/bar_final_json.pkl','rb'))

# Taking care of multiple prices old method but chainer is useful

In [ ]:
def chainer(string):
    return list(chain.from_iterable(string.str.split('/')))


In [ ]:
lengths = str_menu['price'].str.split('/').map(len)

In [ ]:
str_menu = pd.DataFrame({'main_category': np.repeat(str_menu['main_category'], lengths),
                    'sub_category': np.repeat(str_menu['sub_category'], lengths),
                    'name': np.repeat(str_menu['name'],lengths),
                    'description': np.repeat(str_menu['description'],lengths),
                    'price': chainer(str_menu['price'])})

In [ ]:
str_menu.reset_index(inplace=True,drop=True)

In [ ]:
str_menu.loc[45,'price']="250"

In [ ]:
pickle.dump(str_menu,open('half_menu_main.pkl','wb'))

In [ ]:
str_menu=pickle.load(open('half_menu_main.pkl','rb'))

# js.data

In [17]:
pickle.dump(final_json,open('jason.pkl','wb'))

In [72]:
pickle.dump(final_list_json,open('jason.pkl','wb'))

In [71]:
pickle.dump(my_json,open('jason.pkl','wb'))

NameError: name 'my_json' is not defined

In [122]:
my_json={

  "categories": ["bar_bites", "salads", "veg_starters", "non_veg_starters"],
  "subcategories": {
    "bar_bites" : ["bar_bites1"],
    "salads": ["salads1"],
    "veg_starters": ["veg_starter","from_the_clay_oven"],
    "non_veg_starters":["chicken_starters","from_the_clay_oven","lamb_starters","egg_starters","from_the_grill"]
  },
  "items" :{
    "bar_bites1": [ {
      "dish_name": "NACHOS (Veg/Chicken)",
      "description":
      "Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream",
      "price": "190/230"
    },
      {
        "dish_name": "CHEESE MASALA PAPAD",
        "description":
        "Crispy fried Mini Papads Topped With A Tangy Masala Mixture And Grated Cheese",
        "price": "160"
      },
      {
        "dish_name": "H.O.C SIGNATURE MASALA PEANUTS",
        "description":
        "A Favorful Version Of Regular Peanuts Tossed In A Variety of Spices",
        "price": "160"
      }
    ],
    "salads1":[ {
      "dish_name": "GREEN SALAD",
      "description": "Traditional Green Cuts With Mint Yoghurt Dip",
      "price": "170"
    },
      {
        "dish_name": "HOC SALAD (Veg/Chicken)",
        "description":
        "Melange Of Crunchy Lettuce, Bell Peppers And Olives With Our Homemade Dressing",
        "price": "180/210"
      }
    ],
    "veg_starter":[{
      "dish_name": "CHEESE & CORN TRIANGLE (new)",
      "description":
      "Shredded Cheese, Corn & Seasonings Wrapped In Wonton Sheet & Deep Fried",
      "price": "220"
    },
      {
        "dish_name": "CHAMPIGNONS FARCIS (new)",
        "description": "Sauteed Button Mushroom Stuffed With Cheese & Herbs",
        "price": "230"
      }
    ],
    "from_the_clay_oven":[{
      "dish_name": "MAKHMALI PANEER TIKKA",
      "description":
      "Mouth Melting Paneer Cubes Marinated With A Cheese Curd Mixture, Cream And Mild Spices",
      "price": "270"
    },
      {
        "dish_name": "ANGARI PANEER TIKKA",
        "description": "Clay Oven Roasted Spicy Morsels Of Cottage Cheese",
        "price": "270"
      }
    ]
  }
}

In [120]:
my_json={

  "categories": ["bar_bites", "salads", "veg_starters", "non_veg_starters"],
  "subcategories": {
    "bar_bites" : ["bar_bites1"],
    "salads": ["salads1"],
    "veg_starters": ["veg_starter","from_the_clay_oven"],
    "non_veg_starters":["chicken_starters","from_the_clay_oven","lamb_starters","egg_starters","from_the_grill"]
  },
  "items" :{
    "bar_bites1": [ {
      "dish_name": "NACHOS (Veg/Chicken)",
      "description":
      "Crispy Corn Tortillas Topped With Spicy Baked Bean Salsa and Sour Cream",
      "price": "190/230"
    }]
  }
}

In [ ]:
pages[6]=

In [ ]:
pages[7]=

In [ ]:
pages[8]=

In [ ]:
pages[9]=

In [ ]:
pages[10]=

In [ ]:
pages[11]=